In [1]:
# 最后保存这个，这个改了路径
import os 
import csv
import pickle
import numpy as np
import pandas as pd
import get_feature_2 as gf # 获取混合矩阵的特征
from scipy import sparse
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore') #忽略不重要的warning

import Evaluation as Eva # 获取评估函数

In [2]:
import random
random.seed(5)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.adapt import MLkNN
from skmultilearn.ensemble import MajorityVotingClassifier,RakelD
from skmultilearn.cluster import FixedLabelSpaceClusterer
from skmultilearn.problem_transform import BinaryRelevance,ClassifierChain,LabelPowerset

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()


In [3]:
###============= mix + vec =================###
vct_dir = '/home/dqw_lyt/LYT_Task2/New-mixdata/New-doc/vwF_embeddings/'
K = [3,4,5]
W = [3,4,5,6,7]

KW = pd.DataFrame(columns=['k','w'])
for k in K:
    for w in W:
        KW = KW.append({'k': k, 'w': w},ignore_index=True)
        
def concatfet(k=None,w=None):
    if k and w:
        vct = str("X_vWF_{0}_{1}.pkl".format(k,w))
        with open(vct_dir+vct,'rb') as f:
            vctfet = pickle.load(f)
            mixvct = np.concatenate((mixfet,vctfet[0]),axis=1) 
            yield mixvct

def mix_vct(k=None,w=None):
    if k and w:
        mixvct = concatfet(k,w)
        yield from concatfet(k,w)
    else:
        for i in range(KW.shape[0]):
            k = KW.iloc[i,KW.columns.get_loc('k')]
            w = KW.iloc[i,KW.columns.get_loc('w')]
            yield from concatfet(k,w) # 生成器的嵌套可使用yield from

                
                
###============= 生成划分数据集的索引 =================###
# df1 = pd.DataFrame(columns=['Score','Params'])
def splitdata(target,times=1,k=None,w=None):
    rows = mutsites.index # 获取行索引
    np.random.seed(5)
    SEEDs = np.random.randint(200,size=times).tolist()
    # pbar = tqdm(SEEDs,total=times)  
    for SEED in SEEDs:
        # pbar.set_description(str(SEEDs.index(SEED)))
        # pbar.set_postfix(seed=int(SEED))
        trainval_idx,test_idx = train_test_split(rows,test_size=0.2,random_state=SEED)     
       
        # 30D 或14D
#         data = mixfet
#         X_trainval = data[trainval_idx]
#         X_test = data[test_idx]
#         y_trainval = sparse.lil_matrix(target.loc[trainval_idx])
#         y_test = sparse.lil_matrix(target.loc[test_idx])       
#         yield SEED,X_trainval,y_trainval,X_test,y_test
        
        # 94D：mix 与 doc特征拼接
        mv_data = concatfet(k=3,w=5) 
        for data in mv_data:
            X_trainval = data[trainval_idx]
            X_test = data[test_idx]
            y_trainval = sparse.lil_matrix(target.loc[trainval_idx])
            y_test = sparse.lil_matrix(target.loc[test_idx])       
            yield SEED,X_trainval,y_trainval,X_test,y_test

In [4]:
def name(estimator):
    model = str(estimator)
    model_n = model.split('(')[0]
    return model_n

def pretocsv(model_n,y_test,y_score,target_col,test_dir):
    with open(test_dir+'test_'+model_n+'.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(target_col)
        writer.writerows(y_test.toarray())
        writer.writerow(target_col)
        writer.writerows(y_score.toarray())    

# 构建模型

# 1.MLKNN

In [5]:
def mlknn(target,target_col,test_dir,dir_num,model_dir):
    gs_parameters = {'k': range(1,3)}
    gs = GridSearchCV(MLkNN(), gs_parameters,scoring='f1_samples',cv=loo,n_jobs=15)

    times = 5 # 训练验证和测试集划分的次数
    # sp = splitdata(target,times,k=3,w=5) #带doc特征的划分数据集
    sp = splitdata(target,times)
    
    df1 = pd.DataFrame(columns=['Score','Params']) #保存最好的训练结果

    for SEED,X_trainval,y_trainval,X_test,y_test in sp:  
        # print(X_trainval.shape)
        gs.fit(X_trainval,y_trainval)    
        model_n = name(gs.estimator)
        df1 = df1.append({'Score': gs.best_score_, 'Params':gs.best_params_}, ignore_index=True)
      
        mk = MLkNN(**gs.best_params_)
        mk.fit(X_trainval,y_trainval) 
        with open(model_dir+'{0}_{1}.pickle'.format(model_n,SEED),'wb')as f: # model_dir
            pickle.dump(mk,f) #将模型dump进f里面  

        y_score = mk.predict_proba(X_test)
        pretocsv(model_n,y_test,y_score,target_col,test_dir) #* pretocsv(model_n,y_test,y_score,target_col,test_dir)
    df1.to_csv('ISTH_RJ_mmc_result/validation/94D/train_{}.csv'.format(dir_num),mode='a')   #*'result/train_{}.csv'.format(dir_num)    


# 2. BR

In [6]:
def binaryrelevance(target,target_col,test_dir,dir_num,model_dir):
    random.seed(5)
    gs_parameters = [
         {
             'classifier': [SVC(probability=True,random_state=5)],
             'classifier__kernel': ['rbf'],
             'classifier__gamma': [0.01,0.1,1],
             'classifier__C': [1,10,100],
         },
         {
             'classifier':[RandomForestClassifier(oob_score=True,random_state=5)],
             'classifier__n_estimators':np.arange(80,105,1)

         },
    ]

    gs = GridSearchCV(BinaryRelevance(require_dense = [False,True]), gs_parameters,cv=loo,scoring='f1_samples',n_jobs=15)

    times = 5 # 训练验证和测试集划分的次数
    # sp = splitdata(target,times,k=3,w=5)
    sp = splitdata(target,times)
                   
    df1 = pd.DataFrame(columns=['Score','Params']) #保存最好的训练结果

    for SEED,X_trainval,y_trainval,X_test,y_test in sp:  
        gs.fit(X_trainval,y_trainval)    
        model_n = name(gs.estimator)
        df1 = df1.append({'Score': gs.best_score_, 'Params':gs.best_params_}, ignore_index=True)

        br = BinaryRelevance(gs.best_params_['classifier'],require_dense = [False,True])
        br.fit(X_trainval,y_trainval) 
        with open(model_dir+'{0}_{1}.pickle'.format(model_n,SEED),'wb')as f: #python路径要用反斜杠
            pickle.dump(br,f) #将模型dump进f里面  

        y_score = br.predict_proba(X_test)
        pretocsv(model_n,y_test,y_score,target_col,test_dir) #* pretocsv(model_n,y_test,y_score,target_col,test_dir)
        
    df1.to_csv('ISTH_RJ_mmc_result/validation/94D/train_{}.csv'.format(dir_num),mode='a')     


# 3.CC

In [7]:
def classifierchain(target,target_col,test_dir,dir_num,model_dir):
    random.seed(5)
    gs_parameters = [
         {
             'classifier': [SVC(probability=True,random_state=5)],
             'classifier__kernel': ['rbf'],
             'classifier__gamma': [0.001,0.01,0.1],
             'classifier__C': [10,100,1000],
         },
         {
            'classifier':[RandomForestClassifier(oob_score=True,random_state=5)],
            'classifier__n_estimators':np.arange(110,150,1)

         },
    ]

    gs = GridSearchCV(ClassifierChain(require_dense = [False,True]), gs_parameters,cv=loo,scoring='f1_samples',n_jobs=15)

    times = 5 # 训练验证和测试集划分的次数
    # sp = splitdata(target,times,k=3,w=5)
    sp = splitdata(target,times)
                   
    df1 = pd.DataFrame(columns=['Score','Params']) #保存最好的训练结果

    for SEED,X_trainval,y_trainval,X_test,y_test in sp:  
        gs.fit(X_trainval,y_trainval)    
        model_n = name(gs.estimator)
        df1 = df1.append({'Score': gs.best_score_, 'Params':gs.best_params_}, ignore_index=True)
        
        cc = ClassifierChain(gs.best_params_['classifier'],require_dense = [False,True])
        cc.fit(X_trainval,y_trainval) 
        with open(model_dir+'{0}_{1}.pickle'.format(model_n,SEED),'wb')as f: #python路径要用反斜杠
            pickle.dump(cc,f) #将模型dump进f里面  

        y_score = cc.predict_proba(X_test)
        pretocsv(model_n,y_test,y_score,target_col,test_dir) #* pretocsv(model_n,y_test,y_score,target_col,test_dir)
    
    df1.to_csv('ISTH_RJ_mmc_result/validation/94D/train_{}.csv'.format(dir_num),mode='a')     


# 4. LP

In [8]:
def labelpowerset(target,target_col,test_dir,dir_num,model_dir):
    random.seed(5)
    gs_parameters = [
         {
             'classifier': [SVC(probability=True,random_state=5)],
             'classifier__kernel': ['rbf'],
             'classifier__gamma': [0.1,1,10],
             'classifier__C': [1,10,100],
         },
         {
             'classifier':[RandomForestClassifier(oob_score=True,random_state=5)],
             'classifier__n_estimators':np.arange(80,130,1)
         },
    ]

    gs = GridSearchCV(LabelPowerset(require_dense = [False,True]), gs_parameters,cv=loo,scoring='f1_samples',n_jobs=15)


    times = 5 # 训练验证和测试集划分的次数
    # sp = splitdata(target,times,k=3,w=5)
    sp = splitdata(target,times)
                   
    df1 = pd.DataFrame(columns=['Score','Params']) #保存最好的训练结果


    for SEED,X_trainval,y_trainval,X_test,y_test in sp:  
        gs.fit(X_trainval,y_trainval)    
        model_n = name(gs.estimator)
        df1 = df1.append({'Score': gs.best_score_, 'Params':gs.best_params_}, ignore_index=True)

        lp = LabelPowerset(gs.best_params_['classifier'],require_dense = [False,True])
        lp.fit(X_trainval,y_trainval) 
        with open(model_dir+'{0}_{1}.pickle'.format(model_n,SEED),'wb')as f: #python路径要用反斜杠
            pickle.dump(lp,f) #将模型dump进f里面  

        y_score = lp.predict_proba(X_test)
        pretocsv(model_n,y_test,y_score,target_col,test_dir) #* pretocsv(model_n,y_test,y_score,target_col,test_dir)

        
    df1.to_csv('ISTH_RJ_mmc_result/validation/94D/train_{}.csv'.format(dir_num),mode='a')



# 5. RK

In [9]:
def rakeld(target,target_col,test_dir,dir_num,model_dir):
    random.seed(5)
    gs_parameters = [
         {
             'base_classifier': [SVC(probability=True,random_state=5)],
             'base_classifier__kernel': ['rbf'],
             'base_classifier__gamma': [0.1,1,10],
             'base_classifier__C': [1,10,100],
             'labelset_size': [4,5,6],
         },
         {
             'base_classifier':[RandomForestClassifier(oob_score=True,random_state=5)],
             'base_classifier__n_estimators':np.arange(10,60,1),
             'labelset_size': [4,5,6],
         },
    ]

    gs = GridSearchCV(RakelD(base_classifier_require_dense = [False,True]), gs_parameters,cv=loo,scoring='f1_samples',n_jobs=15)

    times = 5 # 训练验证和测试集划分的次数
    # sp = splitdata(target,times,k=3,w=5)
    sp = splitdata(target,times)
                   
    df1 = pd.DataFrame(columns=['Score','Params']) #保存最好的训练结果

    for SEED,X_trainval,y_trainval,X_test,y_test in sp:  
        gs.fit(X_trainval,y_trainval)    
        model_n = name(gs.estimator)
        df1 = df1.append({'Score': gs.best_score_, 'Params':gs.best_params_}, ignore_index=True)

        rk = RakelD(gs.best_params_['base_classifier'],base_classifier_require_dense = [False,True])
        rk.fit(X_trainval,y_trainval) 
        with open(model_dir+'{0}_{1}.pickle'.format(model_n,SEED),'wb')as f: #python路径要用反斜杠
            pickle.dump(rk,f) #将模型dump进f里面  

        y_score = rk.predict_proba(X_test)
        pretocsv(model_n,y_test,y_score,target_col,test_dir) #* pretocsv(model_n,y_test,y_score,target_col,test_dir)

    df1.to_csv('ISTH_RJ_mmc_result/validation/94D/train_{}.csv'.format(dir_num),mode='a')    


# 加载数据

In [10]:
data_dir = '/home/dqw_lyt/LYT_Task2/New-mixdata/'

mutsites_p = data_dir + 'New-2-3.csv'
mutsites = pd.read_csv(mutsites_p)['Amino Acid Substitution']

###============= mix feature =================###
pssm_p = data_dir + 'wt.pssm'
spd33_p = data_dir + 'wt.spd33'
spotd_dir = data_dir + 'New-SPOTD'

mixfet = gf.mix_feature(mutsites,pssm_p,spotd_dir,spd33_p)
print(mixfet.shape)

(528, 30)


In [11]:
datasets = ['New-2-3.csv']
           # ['New-2-0.csv',
           # 'New-2-1.csv',
           # 'New-2-2.csv', 
           # 'New-3-0.csv',
           # 'New-3-1.csv',
           # 'New-3-2.csv',
           # 'New-3-3.csv'] 

data_pbar = tqdm(datasets)
for data_csv in data_pbar:
    data_pbar.set_description(data_csv)
    dir_num = 'f1_loo_'+data_csv[-7:-6]+'_'+data_csv[-5:-4]
    
    model_p = '/home/dqw_lyt/LYT_Task2/script/ISTH_RJ_mmc_result/model/94D/' 
    model_dir = model_p+dir_num+'/'
    
    test_p = '/home/dqw_lyt/LYT_Task2/script/ISTH_RJ_mmc_result/predict/94D/'
    test_dir = test_p+dir_num+'/'
    
    target_p = data_dir + data_csv
    target_df = pd.read_csv(target_p)
    target = target_df.drop(columns=['Amino Acid Substitution'])
    target_col = list(target.columns)
    
    if os.path.isdir(model_p):
        os.mkdir(os.path.join(model_p, dir_num))
    if os.path.isdir(test_p):
        os.mkdir(os.path.join(test_p, dir_num))       
    
    ###============= training and prediction =================###
    mlknn(target,target_col,test_dir,dir_num,model_dir)
    
    binaryrelevance(target,target_col,test_dir,dir_num,model_dir)
    
    classifierchain(target,target_col,test_dir,dir_num,model_dir)
  
    labelpowerset(target,target_col,test_dir,dir_num,model_dir)
  
    rakeld(target,target_col,test_dir,dir_num,model_dir)

    tqdm.write(data_csv)

New-2-3.csv:   0%|          | 0/1 [00:00<?, ?it/s]

(422, 94)
(422, 94)
(422, 94)
(422, 94)
(422, 94)


New-2-3.csv: 100%|██████████| 1/1 [4:54:18<00:00, 17658.33s/it]

New-2-3.csv
